In [91]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, RNN
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [110]:
data_path = '../data/preprocessed_data_2010_2025.csv'

In [111]:
df = pd.read_csv(data_path)

In [112]:
df.head()

,Datetime,Temperature,Condition,Wind Direction,Wind Speed
0,2010-01-01 00:00:00,10.000000,Fair,240.0,7.0
1,2010-01-01 01:00:00,10.000000,Fair,270.0,6.0
2,2010-01-01 02:00:00,10.000000,Fair,250.0,7.0
3,2010-01-01 03:00:00,10.000000,Fair,250.0,7.0
4,2010-01-01 04:00:00,11.111111,Fair,250.0,7.0


In [113]:
df.rename(columns={'Condiction' : 'Condition'},inplace=True)

In [114]:
df_ = pd.get_dummies(data=df['Condition'])

In [116]:
df.loc[43372]

Datetime          2015-01-08 04:00:00
Temperature                  1.111111
Condition                     Drizzle
Wind Direction                  270.0
Wind Speed                       15.0
Name: 43372, dtype: object

In [122]:
df = pd.get_dummies(df, columns=['Condition'], drop_first=True)

In [124]:
df['Datetime'] = pd.to_datetime(df['Datetime'])

In [126]:
df = df.sort_values('Datetime')

In [127]:
df = df.set_index('Datetime')

In [128]:
df.head()

,Temperature,Wind Direction,Wind Speed,Condition_Blowing Sand,Condition_Cloudy,Condition_Drizzle,Condition_Duststorm,Condition_Fair,Condition_Fog,Condition_Freezing Rain,Condition_Hail,Condition_Haze,Condition_Heavy Drizzle,Condition_Heavy Rain,Condition_Heavy Snow,Condition_Light Drizzle,Condition_Light Rain,Condition_Light Snow,Condition_Mist,Condition_Mostly Cloudy,Condition_Partial Fog,Condition_Partly Cloudy,Condition_Rain,Condition_Rain Shower,Condition_Sand,Condition_Sandstorm,Condition_Sleet,Condition_Small Hail,Condition_Smoke,Condition_Snow,Condition_Snow Grains,Condition_Squalls,Condition_Thunder,Condition_Widespread Dust,Condition_Wintry Mix
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-01-01 00:00:00,10.000000,240.0,7.0,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2010-01-01 01:00:00,10.000000,270.0,6.0,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2010-01-01 02:00:00,10.000000,250.0,7.0,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2010-01-01 03:00:00,10.000000,250.0,7.0,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2010-01-01 04:00:00,11.111111,250.0,7.0,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [132]:
df2.loc[0]['Wind Direction']

NameError: name 'df2' is not defined

In [129]:
df['WindDir_sin'] = np.sin(np.deg2rad(df['Wind Direction']))
df['WindDir_cos'] = np.cos(np.deg2rad(df['Wind Direction']))
df.drop(columns=['Wind Direction'],inplace=True)

In [130]:
df.head()

,Temperature,Wind Speed,Condition_Blowing Sand,Condition_Cloudy,Condition_Drizzle,Condition_Duststorm,Condition_Fair,Condition_Fog,Condition_Freezing Rain,Condition_Hail,Condition_Haze,Condition_Heavy Drizzle,Condition_Heavy Rain,Condition_Heavy Snow,Condition_Light Drizzle,Condition_Light Rain,Condition_Light Snow,Condition_Mist,Condition_Mostly Cloudy,Condition_Partial Fog,Condition_Partly Cloudy,Condition_Rain,Condition_Rain Shower,Condition_Sand,Condition_Sandstorm,Condition_Sleet,Condition_Small Hail,Condition_Smoke,Condition_Snow,Condition_Snow Grains,Condition_Squalls,Condition_Thunder,Condition_Widespread Dust,Condition_Wintry Mix,WindDir_sin,WindDir_cos
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-01-01 00:00:00,10.000000,7.0,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,-0.866025,-5.000000e-01
2010-01-01 01:00:00,10.000000,6.0,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,-1.000000,-1.836970e-16
2010-01-01 02:00:00,10.000000,7.0,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,-0.939693,-3.420201e-01
2010-01-01 03:00:00,10.000000,7.0,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,-0.939693,-3.420201e-01
2010-01-01 04:00:00,11.111111,7.0,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,-0.939693,-3.420201e-01
